# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

The corporate office of a growing chain of coffee shops is looking to expand. They plan to open a new store in Chicago, IL, but cannot determine the most optimal location for their new store. The goal of this project will be to identify the most optimal location for a new coffee shop based on trends in location data.

## Data <a name="data"></a>

I will call the Foursqaure API to fetch this data. All of my analysis will be based on indicators driven from the Foursqaure dataset. Some factors to keep in mind when selecting what data to use in analysis for this use case:

    Number of competing coffee shops in each area (we will use zip codes).
    Type of attractions adjacent to established, successful coffee shops.



We will begin by exploring our zip codes and differences between cities

In [79]:
import pandas as pd, numpy as np

# read in all U.S. zips
zips_df = pd.read_csv(r'C:\Users\jackh\Desktop\projects\IBM-Applied-Data-Science-Capstone\data\uszips.csv')

# filter down to our city
zips_df = zips_df.loc[zips_df['city'].isin(['Chicago']) & zips_df['state_id'].isin(['IL'])]

# select the columns we need
zips_df = zips_df[['zip', 'lat', 'lng', 'city', 'state_id', 'state_name']].reset_index()

# show number of zips
print(zips_df.groupby(['city', 'state_id'], as_index=False).count().rename(columns={'zip': 'zip_count'}).drop(columns=['lat', 'lng', 'state_name']).sort_values('zip_count', ascending=False))

        city state_id  index  zip_count
1    Chicago       IL     56         56
0    Atlanta       GA     40         40
2  Nashville       TN     24         24


As we can see, Chicago has the most zip codes, and is the largest city of the three. Let's work with the Chicago zips first, and determine the most optimal Chicago location. Once we have completed that process for all three cities, we will compare the final three candidates.

In [80]:
# first, let's geocode Chicago to define our map view
from geopy.geocoders import Nominatim

address = 'Chicago, IL, USA'

geolocator = Nominatim(user_agent='chicago_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'Latitude: {latitude}, Longitude: {longitude}')

Latitude: 41.8755616, Longitude: -87.6244212


In [81]:
# now, we visualize the zips
import folium

# create chicago-only dataframe
chicago_df = zips_df[zips_df['city'].str.contains('Chicago')]

# initaiate map with view of Chicago
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers for each zip
for lat, lng, z in zip(chicago_df['lat'], chicago_df['lng'], chicago_df['zip']):
    label = '{}'.format(z)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_chicago)

map_chicago

Looks good. Next, we will start calling Foursquare's API to begin analyzing venue data, so that we can apply some clustering methods.

In [82]:
CLIENT_ID = 'IOSOZCWY3TE0ZFLCXGC3NDBVYPRM4CLR2SGZUDSXFNHI2S1B' # your Foursquare ID
CLIENT_SECRET = '4Y4F3QXQ0CA2VSQLL2OLOVXLDRHRSTXKMUD04RQPZKTJXO1N' # your Foursquare Secret
ACCESS_TOKEN = 'SS0UU3CPORXILGWUOB1WNAKWEW0HS2BCIWU1SN4KXHHND1L5' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

Define a function to retrieve nearby venues for each zip from the Foursqaure database.

In [83]:
import json, requests


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(f'fetching venues in zip code: {name}')
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip', 
                  'Zip Latitude', 
                  'Zip Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [84]:
chicago_venues = getNearbyVenues(names=chicago_df['zip'], latitudes=chicago_df['lat'], longitudes=chicago_df['lng'])
chicago_venues

fetching venues in zip code: 60601
fetching venues in zip code: 60602
fetching venues in zip code: 60603
fetching venues in zip code: 60604
fetching venues in zip code: 60605
fetching venues in zip code: 60606
fetching venues in zip code: 60607
fetching venues in zip code: 60608
fetching venues in zip code: 60609
fetching venues in zip code: 60610
fetching venues in zip code: 60611
fetching venues in zip code: 60612
fetching venues in zip code: 60613
fetching venues in zip code: 60614
fetching venues in zip code: 60615
fetching venues in zip code: 60616
fetching venues in zip code: 60617
fetching venues in zip code: 60618
fetching venues in zip code: 60619
fetching venues in zip code: 60620
fetching venues in zip code: 60621
fetching venues in zip code: 60622
fetching venues in zip code: 60623
fetching venues in zip code: 60624
fetching venues in zip code: 60625
fetching venues in zip code: 60626
fetching venues in zip code: 60628
fetching venues in zip code: 60629
fetching venues in z

,Zip,Zip Latitude,Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,60601,41.88526,-87.62194,Wildberry Pancakes & Cafe,41.884412,-87.623047,Breakfast Spot
1,60601,41.88526,-87.62194,Giordano's,41.885130,-87.623761,Pizza Place
2,60601,41.88526,-87.62194,sweetgreen,41.884964,-87.624728,Salad Place
3,60601,41.88526,-87.62194,Harris Theatre for Music and Dance,41.883883,-87.621992,Theater
4,60601,41.88526,-87.62194,"Radisson Blu Aqua Hotel, Chicago",41.886308,-87.619921,Hotel
...,...,...,...,...,...,...,...
1722,60661,41.88309,-87.64401,Sarpino's Pizza,41.885724,-87.643808,Pizza Place
1723,60661,41.88309,-87.64401,Paddy O'Fegan's,41.885880,-87.647419,Bar
1724,60661,41.88309,-87.64401,Lake & Union Tap And Grill,41.885669,-87.645304,Bar
1725,60661,41.88309,-87.64401,Brideside Chicago Bridesmaid Dress Store,41.886000,-87.645110,Bridal Shop


Let's check how many venues were returned per zip code

In [85]:
chicago_venues.groupby('Zip', as_index=False).count()

,Zip,Zip Latitude,Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,60601,100,100,100,100,100,100
1,60602,100,100,100,100,100,100
2,60603,100,100,100,100,100,100
3,60604,100,100,100,100,100,100
4,60605,62,62,62,62,62,62
5,60606,100,100,100,100,100,100
6,60607,56,56,56,56,56,56
7,60608,17,17,17,17,17,17
8,60609,12,12,12,12,12,12
9,60610,72,72,72,72,72,72


Now, let's check how many unique venue categories were returned

In [86]:
print('There are {} unique categories.'.format(len(chicago_venues['Venue Category'].unique())))

There are 268 unique categories.


In [87]:
# one hot encoding
chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chicago_onehot['Zip'] = chicago_venues['Zip'] 

# move neighborhood column to the first column
fixed_columns = [chicago_onehot.columns[-1]] + list(chicago_onehot.columns[:-1])
chicago_onehot = chicago_onehot[fixed_columns]

chicago_onehot.head()

,Zip,ATM,Adult Boutique,Airport,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Arcade,...,Video Store,Vietnamese Restaurant,Waste Facility,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,60601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,60601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,60601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,60601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,60601,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean frequency of the frequency of occurrence of each category

In [88]:
chicago_grouped = chicago_onehot.groupby('Zip').mean().reset_index()
chicago_grouped

,Zip,ATM,Adult Boutique,Airport,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Arcade,...,Video Store,Vietnamese Restaurant,Waste Facility,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,60601,0.0,0.000000,0.000000,0.0,0.040000,0.01,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000
1,60602,0.0,0.000000,0.000000,0.0,0.030000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000
2,60603,0.0,0.000000,0.000000,0.0,0.020000,0.01,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
3,60604,0.0,0.000000,0.000000,0.0,0.020000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
4,60605,0.0,0.000000,0.000000,0.0,0.032258,0.00,0.000000,0.080645,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129
5,60606,0.0,0.000000,0.000000,0.0,0.030000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.020000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,60607,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,60608,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,...,0.058824,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,60609,0.0,0.000000,0.000000,0.0,0.166667,0.00,0.000000,0.000000,0.083333,...,0.000000,0.000000,0.083333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,60610,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.013889,0.013889,0.000000,0.000000


Let's print the top 5 most frequented venues for each zip code.

In [89]:
num_top_venues = 5

for z in chicago_grouped['Zip']:
    print("----"+str(z)+"----")
    temp = chicago_grouped[chicago_grouped["Zip"] == z].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----60601----
                 venue  freq
0                Hotel  0.10
1   Seafood Restaurant  0.05
2            Hotel Bar  0.04
3          Coffee Shop  0.04
4  American Restaurant  0.04


----60602----
            venue  freq
0           Hotel  0.09
1         Theater  0.06
2  Sandwich Place  0.05
3     Coffee Shop  0.05
4  Clothing Store  0.04


----60603----
                venue  freq
0         Coffee Shop  0.10
1      Sandwich Place  0.04
2  Italian Restaurant  0.03
3              Museum  0.03
4             Theater  0.03


----60604----
                venue  freq
0         Coffee Shop  0.08
1               Hotel  0.06
2  Italian Restaurant  0.05
3      Sandwich Place  0.04
4         Pizza Place  0.04


----60605----
              venue  freq
0    History Museum  0.15
1          Aquarium  0.08
2            Museum  0.05
3    Sandwich Place  0.05
4  Sculpture Garden  0.03


----60606----
                      venue  freq
0               Coffee Shop  0.09
1            Sandwich Place 

Now we will create a new dataframe with 10 most common venues for each zip code.

In [90]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [91]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zip']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
zips_venues_sorted = pd.DataFrame(columns=columns)
zips_venues_sorted['Zip'] = chicago_grouped['Zip']

for ind in np.arange(chicago_grouped.shape[0]):
    zips_venues_sorted.iloc[ind, 1:] = return_most_common_venues(chicago_grouped.iloc[ind, :], num_top_venues)

zips_venues_sorted.head()

,Zip,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,60601,Hotel,Seafood Restaurant,Hotel Bar,Coffee Shop,American Restaurant,Sandwich Place,Park,Theater,Grocery Store,Gym
1,60602,Hotel,Theater,Sandwich Place,Coffee Shop,Clothing Store,American Restaurant,Snack Place,Italian Restaurant,Salad Place,Donut Shop
2,60603,Coffee Shop,Sandwich Place,Italian Restaurant,Museum,Theater,Burger Joint,Sushi Restaurant,Park,Hotel,Snack Place
3,60604,Coffee Shop,Hotel,Italian Restaurant,Sandwich Place,Pizza Place,Asian Restaurant,Museum,Theater,Burger Joint,Snack Place
4,60605,History Museum,Aquarium,Museum,Sandwich Place,Sculpture Garden,Breakfast Spot,Historic Site,Gift Shop,American Restaurant,Recording Studio


Clustering Neighborhoods Based on Venue Categories

In [92]:
from sklearn.cluster import KMeans

kclusters = 15

chicago_grouped_clustering = chicago_grouped.drop('Zip', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chicago_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([ 0,  0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  8,  0,  0, 14,  0,  0,
        0,  8,  3])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [93]:
# add clustering labels
zips_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

chicago_merged = chicago_df.rename(columns={'zip': 'Zip'})

# merge dt_toronto_grouped with dt_toronto_df to add latitude/longitude for each neighborhood
chicago_merged = chicago_merged.join(zips_venues_sorted.set_index('Zip'), on='Zip')

chicago_merged.head() # check the last columns!

,index,Zip,lat,lng,city,state_id,state_name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,20691,60601,41.88526,-87.62194,Chicago,IL,Illinois,0,Hotel,Seafood Restaurant,Hotel Bar,Coffee Shop,American Restaurant,Sandwich Place,Park,Theater,Grocery Store,Gym
65,20692,60602,41.88309,-87.62912,Chicago,IL,Illinois,0,Hotel,Theater,Sandwich Place,Coffee Shop,Clothing Store,American Restaurant,Snack Place,Italian Restaurant,Salad Place,Donut Shop
66,20693,60603,41.88022,-87.62549,Chicago,IL,Illinois,0,Coffee Shop,Sandwich Place,Italian Restaurant,Museum,Theater,Burger Joint,Sushi Restaurant,Park,Hotel,Snack Place
67,20694,60604,41.87814,-87.62837,Chicago,IL,Illinois,0,Coffee Shop,Hotel,Italian Restaurant,Sandwich Place,Pizza Place,Asian Restaurant,Museum,Theater,Burger Joint,Snack Place
68,20695,60605,41.86684,-87.61983,Chicago,IL,Illinois,0,History Museum,Aquarium,Museum,Sandwich Place,Sculpture Garden,Breakfast Spot,Historic Site,Gift Shop,American Restaurant,Recording Studio


Check how many zips per cluster

In [94]:
print(chicago_merged.groupby('Cluster Labels', as_index=False).count())

    Cluster Labels  index  Zip  lat  lng  city  state_id  state_name  \
0                0     29   29   29   29    29        29          29   
1                1      1    1    1    1     1         1           1   
2                2      1    1    1    1     1         1           1   
3                3      1    1    1    1     1         1           1   
4                4      1    1    1    1     1         1           1   
5                5      1    1    1    1     1         1           1   
6                6      1    1    1    1     1         1           1   
7                7      1    1    1    1     1         1           1   
8                8     12   12   12   12    12        12          12   
9                9      2    2    2    2     2         2           2   
10              10      1    1    1    1     1         1           1   
11              11      1    1    1    1     1         1           1   
12              12      2    2    2    2     2         2        

Finally, let's visualize our clusters!

In [96]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chicago_merged['lat'], chicago_merged['lng'], chicago_merged['Zip'], chicago_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Based on these clustering results, we can see that the most common cluster label for successful coffee shops would be 0. Let's look for the cluster 0 zip codes with the lowest coffee shop density among the top 10 venues.

In [99]:
print('Cluster 0 includes', len(chicago_merged[chicago_merged['Cluster Labels']==0]), 'zip codes')

Cluster 0 includes 29 zip codes


In [116]:
chicago_candidates = chicago_merged[chicago_merged['Cluster Labels']==0]

for col in chicago_candidates.columns:
    if 'Most Common Venue' in col:
        chicago_candidates = chicago_candidates.loc[chicago_candidates[col]!='Coffee Shop']

print(len(chicago_candidates), 'candidates with no competing coffee shops')

14 candidates with no competing coffee shops


Now, let's plot our remaining 14 candidates

In [117]:
# initaiate map with view of Chicago
map_candidates = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers for each zip
for lat, lng, z in zip(chicago_candidates['lat'], chicago_candidates['lng'], chicago_candidates['Zip']):
    label = '{}'.format(z)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_candidates)

map_candidates

From this folium plot, we can select the most optimal candidates based on geographic location. Finally, we will manually analyze nearby attractions of our final 14 zip codes to determine the best solution to our site selection project. 

In [119]:
chicago_candidates = chicago_candidates[chicago_candidates['Zip'].isin([60605, 60616, 60642, 60622, 60640, 60618])]
chicago_candidates

,index,Zip,lat,lng,city,state_id,state_name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,20695,60605,41.86684,-87.61983,Chicago,IL,Illinois,0,History Museum,Aquarium,Museum,Sandwich Place,Sculpture Garden,Breakfast Spot,Historic Site,Gift Shop,American Restaurant,Recording Studio
79,20706,60616,41.84525,-87.62725,Chicago,IL,Illinois,0,Gym / Fitness Center,Storage Facility,Restaurant,New American Restaurant,Bus Line,Bus Station,Seafood Restaurant,Chinese Restaurant,Park,Light Rail Station
81,20708,60618,41.94700,-87.70244,Chicago,IL,Illinois,0,Park,Theater,Pub,Breakfast Spot,Bank,Cosmetics Shop,Big Box Store,Mobile Phone Shop,Gym,Supplement Shop
85,20712,60622,41.90275,-87.68330,Chicago,IL,Illinois,0,Bar,Italian Restaurant,Sandwich Place,Mediterranean Restaurant,Bakery,Pet Store,Pizza Place,Taco Place,Steakhouse,Sports Bar
101,20728,60640,41.97237,-87.66347,Chicago,IL,Illinois,0,Vietnamese Restaurant,Chinese Restaurant,Thai Restaurant,Grocery Store,Ice Cream Shop,Gift Shop,Bike Rental / Bike Share,Mexican Restaurant,Middle Eastern Restaurant,Historic Site
103,20730,60642,41.90161,-87.65803,Chicago,IL,Illinois,0,Bar,Indie Movie Theater,Seafood Restaurant,Auto Workshop,Auto Dealership,History Museum,Art Gallery,Kitchen Supply Store,Non-Profit,Noodle House


## Conclusion

Due to the prime location in the heart of downtown Chicago, a low number of competitive coffee shops, and a vast number of high-traffic nearby venues, we should recommend that the new location be opened in zip code 60605.